Analyze GPU scan parquet operator in Spark event logs.

In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd

cpu_dir = "/home/rou/src/spark-rapids-tools/CPU-3k-new"
gpu_dir = "/home/rou/src/spark-rapids-tools/GPU-3k-new"
cpu_files = os.listdir(cpu_dir)
cpu_files.sort()
gpu_files = os.listdir(gpu_dir)
gpu_files.sort()

In [2]:
gpu_scan_times = []
for i, app in enumerate(gpu_files):
    sql_info = pd.read_csv(gpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "scan time") & (sql_info["nodeName"] == "GpuScan parquet ")].copy()
    filtered_df["appIndex"] = i 
    filtered_df["app"] = app 
    filtered_df.drop_duplicates(subset=["accumulatorId"], inplace=True)
    gpu_scan_times.append(filtered_df) 
gpu_scan_times_df = pd.concat(gpu_scan_times)
gpu_scan_times_df[gpu_scan_times_df["appIndex"] == 3]

,appIndex,sqlID,nodeID,nodeName,accumulatorId,name,min,median,max,total,metricType,stageIds,app
150,3,24,39,GpuScan parquet,1480,scan time,0,1,655,6465,timing,36,app-20220210142923-0431
181,3,24,45,GpuScan parquet,1514,scan time,0,448,1547,91984,timing,46,app-20220210142923-0431
203,3,24,49,GpuScan parquet,1538,scan time,0,62,62,62,timing,31,app-20220210142923-0431
223,3,24,53,GpuScan parquet,1559,scan time,0,52,52,52,timing,33,app-20220210142923-0431
305,3,24,71,GpuScan parquet,1653,scan time,0,514,1044,80903,timing,35,app-20220210142923-0431
327,3,24,75,GpuScan parquet,1677,scan time,0,57,57,57,timing,32,app-20220210142923-0431
347,3,24,79,GpuScan parquet,1698,scan time,0,55,55,55,timing,34,app-20220210142923-0431
435,3,24,100,GpuScan parquet,1799,scan time,0,284,874,64582,timing,44,app-20220210142923-0431
524,3,24,119,GpuScan parquet,1900,scan time,0,268,871,53209,timing,40,app-20220210142923-0431
616,3,24,140,GpuScan parquet,2005,scan time,0,118,265,19605,timing,38,app-20220210142923-0431


In [3]:
cpu_scan_times = []
for i, app in enumerate(cpu_files) :
    sql_info = pd.read_csv(cpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "scan time") & (sql_info["nodeName"] == "Scan parquet ")].copy()
    filtered_df["appIndex"] = i  
    filtered_df["app"] = app 
    # filtered_df.drop_duplicates(subset=["accumulatorId"], inplace=True)
    cpu_scan_times.append(filtered_df)
cpu_scan_times_df = pd.concat(cpu_scan_times)
cpu_scan_times_df[cpu_scan_times_df["appIndex"] == 14]

,appIndex,sqlID,nodeID,nodeName,accumulatorId,name,min,median,max,total,metricType,stageIds,app
64,14,24,24,Scan parquet,2791,scan time,0,38,370,44548,timing,57,app-20220208014431-0190
84,14,24,32,Scan parquet,2811,scan time,0,663,663,663,timing,41,app-20220208014431-0190
95,14,24,38,Scan parquet,2822,scan time,0,342,342,342,timing,31,app-20220208014431-0190
125,14,24,49,Scan parquet,2842,scan time,0,613,737,4241,timing,35,app-20220208014431-0190
179,14,24,68,Scan parquet,2880,scan time,0,60,1349,135198,timing,54,app-20220208014431-0190
199,14,24,76,Scan parquet,2900,scan time,0,723,723,723,timing,37,app-20220208014431-0190
213,14,24,81,Scan parquet,2914,scan time,0,603,695,4587,timing,34,app-20220208014431-0190
227,14,24,87,Scan parquet,2900,scan time,0,723,723,723,timing,37,app-20220208014431-0190
252,14,24,98,Scan parquet,2929,scan time,0,50,1298,311951,timing,52,app-20220208014431-0190
270,14,24,103,Scan parquet,2947,scan time,0,573,828,4824,timing,36,app-20220208014431-0190


In [4]:
gpu_scan_counts = gpu_scan_times_df.groupby("appIndex").size().reset_index(name="counts gpu")
cpu_scan_counts = cpu_scan_times_df.groupby("appIndex").size().reset_index(name="counts cpu")
combined_counts = pd.merge(gpu_scan_counts, cpu_scan_counts, on="appIndex")
combined_counts[combined_counts["counts gpu"] != combined_counts["counts cpu"]]

,appIndex,counts gpu,counts cpu
0,0,5,6
4,4,11,12
5,5,7,8
6,6,5,6
7,7,6,7
...,...,...,...
93,94,7,8
94,95,4,5
97,98,5,6
99,100,3,4


2 - query 24a:
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` = 7
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` = 8
3 - query 24b: GPU misisng 1 `ReusedExchange`
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` = 7
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` = 8
5 - query 47:
  CPU: 15 `Scan parquet` - 11 `ReusedExchange` = 4
  GPU: 15 `GpuScan parquet` - 10 `ReusedExchange` = 5
6 - query 56:
  CPU: 18 `Scan parquet` - 9 `ReusedExchange` - 2 `ReusedSubquery` = 7
  GPU: 18 `GpuScan parquet` - 9 `ReusedExchange` - 2 `ReusedSubquery` = 7 (why 8???)
7 - query 66:
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` - 1 `ReusedSubquery` = 6
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` - 1 `ReusedSubquery` = 7
8 - query 75:
  CPU: 30 `Scan parquet` - 14 `ReusedExchange` - 4 `ReusedSubquery` = 12
  GPU: 30 `GpuScan parquet` - 12 `ReusedExchange` - 4 `ReusedSubquery` = 14  
 

14: query 14b:
  CPU: 82 `Scan parquet` - 54 `ReusedExchange` - 20 `ReusedSubquery` + 9 nested reuse = 17
  GPU: 82 `GpuScan parquet` - 64 `ReusedExchange` - 11 `ReusedSubquery` + 3 nested reuse = 10???